## Importing Libraries

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np 
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import activations
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation, Add, AveragePooling2D, BatchNormalization, Conv2D, Conv2DTranspose, Dense, Flatten, Input, Reshape, ZeroPadding2D
from tensorflow.keras.applications import ResNet101
from PIL import Image

from google.colab import drive

## Initializing GPU Runtime

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Creating Required Layer Blocks

In [ ]:
def dres_conv(x, s, filters):
    # here the input size changes
    x_skip = x
    f1, f2 = filters

    # third block
    x = Conv2DTranspose(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    # second block
    x = Conv2DTranspose(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    # third block
    x = Conv2DTranspose(f1, kernel_size=(1, 1), strides=(s, s), padding='valid')(x)
    # when s = 2 then it is like downsizing the feature map
    x = BatchNormalization()(x)

    # shortcut 
    x_skip = Conv2DTranspose(f1, kernel_size=(1, 1), strides=(s, s), padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)

    # add 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

In [ ]:
def dres_identity(x, filters): 
    # resnet block where dimension doesnot change.
    # The skip connection is just simple identity conncection
    # There will be 3 blocks and then input will be added

    x_skip = x # this will be used for addition with the residual block 
    f1, f2 = filters

    # first block 
    x = Conv2DTranspose(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)


    # second block # bottleneck (but size kept same with padding)
    x = Conv2DTranspose(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    # third block activation used after adding the input
    x = Conv2DTranspose(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
    x = BatchNormalization()(x)

    # add the input 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

## Loading Data

In [ ]:
(x_train, _), (x_test, _) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
print(f"Shape of x_train: {x_train.shape}")
print(f"Shape of x_test: {x_test.shape}")

170500096/170498071 [==============================] - 4s 0us/step
Shape of x_train: (50000, 32, 32, 3)
Shape of x_test: (10000, 32, 32, 3)


## Creating Encoder

In [ ]:
input_im = Input(shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))
Encoder = ResNet101(include_top=False, weights='imagenet', input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))
x = Encoder(input_im)
x = Flatten()(x)
encoding = Dense(2048, kernel_initializer='he_normal')(x)
encoder = tf.keras.Model(inputs=input_im, outputs=encoding, name='Encoder')

171450368/171446536 [==============================] - 1s 0us/step


## Creating Decoder

In [ ]:
# Decoder
dec_input = Input(shape=(2048,))
x = Dense(2 * 2 * 2048, kernel_initializer='he_normal')(dec_input)
x = Reshape((2, 2, 2048))(x)

x = dres_conv(x, s=2, filters=(512, 2048))
x = dres_identity(x, filters=(512, 2048))
x = dres_identity(x, filters=(512, 2048))

x = dres_conv(x, s=2, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))
x = dres_identity(x, filters=(256, 1024))


x = dres_conv(x, s=2, filters=(128, 512))
x = dres_identity(x, filters=(128, 512))
x = dres_identity(x, filters=(128, 512))
x = dres_identity(x, filters=(128, 512))

x = dres_conv(x, s=1, filters=(64, 256))
x = dres_identity(x, filters=(64, 256))
x = dres_identity(x, filters=(64, 256))
x = Conv2DTranspose(3, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation(activations.sigmoid)(x)
decoder = tf.keras.Model(inputs=dec_input, outputs=decoded, name='Decoder')

## Creating Auto Encoder


In [ ]:
enc_input = Input(shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))
encoding = encoder(enc_input)
decoded = decoder(encoding)
auto_encoder = tf.keras.Model(inputs=enc_input, outputs=decoded, name='AutoEncoder')
auto_encoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
auto_encoder.summary()

Model: "AutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
Encoder (Model)              (None, 2048)              46854528  
_________________________________________________________________
Decoder (Model)              (None, 32, 32, 3)         118988815 
Total params: 165,843,343
Trainable params: 165,638,537
Non-trainable params: 204,806
_________________________________________________________________


## Training and Saving Model
Skip this section if you want to simply load the pretrained model

In [ ]:
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
auto_encoder.fit(x_train, x_train, epochs=10, batch_size=128, shuffle=True, validation_data=(x_test, x_test), callbacks=tensorboard_callback)

Epoch 1/10
391/391 [==============================] - 355s 909ms/step - loss: 0.0366 - val_loss: 0.0750
Epoch 2/10
391/391 [==============================] - 353s 904ms/step - loss: 0.0270 - val_loss: 0.0471
Epoch 3/10
391/391 [==============================] - 352s 901ms/step - loss: 0.0205 - val_loss: 0.0219
Epoch 4/10
391/391 [==============================] - 351s 897ms/step - loss: 0.0186 - val_loss: 0.0186
Epoch 5/10
391/391 [==============================] - 351s 898ms/step - loss: 0.0173 - val_loss: 0.0206
Epoch 6/10
391/391 [==============================] - 352s 900ms/step - loss: 0.0161 - val_loss: 0.0183
Epoch 7/10
391/391 [==============================] - 353s 902ms/step - loss: 0.0150 - val_loss: 0.0164
Epoch 8/10
391/391 [==============================] - 353s 903ms/step - loss: 0.0144 - val_loss: 0.0170
Epoch 9/10
391/391 [==============================] - 353s 902ms/step - loss: 0.0153 - val_loss: 0.0195
Epoch 10/10
391/391 [==============================] - 352s 900m

In [ ]:
# %tensorboard --logdir logs
# Uncomment and run the above line to start TensorBoard GUI

In [ ]:
drive.mount('/content/gdrive', force_remount=True)
ENC_STORE_PATH = "/content/gdrive/My Drive/Colab/TF_ResNet101_ENC.h5"
DEC_STORE_PATH = "/content/gdrive/My Drive/Colab/TF_ResNet101_DEC.h5"
AE_STORE_PATH = "/content/gdrive/My Drive/Colab/TF_ResNet101_AE.h5"
encoder.save_weights(ENC_STORE_PATH)
decoder.save_weights(DEC_STORE_PATH)
auto_encoder.save_weights(AE_STORE_PATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
